In [14]:
# Import
!pip3 install pickle5
!pip install keras-tuner
import pickle5 as pc

from google.colab import drive
drive.mount('/content/drive')


# Import
import numpy as np
import pandas as pd
import json
#from mpi4py import MPI
import random

# import keras
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.initializers import *
from kerastuner.tuners import *
from kerastuner import HyperModel
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier


from sklearn.model_selection import GridSearchCV

from datetime import datetime

# set random seed
np.random.seed(1)
tf.random.set_seed(1)


#prepare forecasting data
def gen_X_sequence(id_df, seq_length, seq_cols, timesteps_pred, type_data = None):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """

    ind_start = 0
    
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0+ind_start, num_elements-seq_length+1-timesteps_pred), range(seq_length+ind_start, num_elements+1-timesteps_pred)):
        yield data_array[start:stop, :]
 

def gen_Y_sequence(id_df, seq_length, seq_cols, timesteps_pred, type_data = None):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """

    ind_start = 0
    
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0+ind_start, num_elements-seq_length+1-timesteps_pred), range(seq_length+ind_start, num_elements+1-timesteps_pred)):
        yield data_array[stop-1, :]

   
def get_dataset(sequence_length, batch_size):
    # set folder path
    fd='/content/drive/MyDrive/PIR_perso'
    fk='/data'

    # import data
    with open(fd + fk + '/data_train_v1', "rb") as fh:
      data_train_df = pc.load(fh).reset_index().iloc[:,1:]
    with open(fd + fk +'/data_test_v1', "rb") as fh:
      data_test_df = pc.load(fh).reset_index().iloc[:,1:]


    # data_train_df = pd.read_pickle(fd_km + '/data_train_v1').reset_index().iloc[:,1:] #full set
    # data_test_df = pd.read_pickle(fd_km + '/data_test_v1').reset_index().iloc[:,1:]

    # create bins
    l = 0.5
    nb_bins = 20 # including one extra bin for RUL>upper_bin_bound
    lower_bin_bound = 0
    upper_bin_bound = 80000

    bins = np.linspace(lower_bin_bound**l, upper_bin_bound**l, nb_bins)**(1/l)
    bins = np.append(bins, data_train_df.RUL.max())

    labels=[i for i in range(bins.shape[0]-1)]

    # categorise data
    data_train_df['RUL_bin'] = pd.cut(data_train_df['RUL'], bins=bins, labels=labels)
    data_test_df['RUL_bin'] = pd.cut(data_test_df['RUL'], bins=bins, labels=labels)

    # build data sequences
    #data_train_group = [data_train_df for _, data_train_df in data_train_df.groupby('ID')]
    #random.shuffle(data_train_group)
    #data_train_df_random = pd.concat(data_train_group)

    data_train = data_train_df[data_train_df.ID <= 500]
    data_val = data_train_df[data_train_df.ID > 9800]
    data_test = data_test_df

    #prepare data
    seq_cols = ['gauge'+str(i) for i in range(1,4)]
    seq_cols1 = ['RUL_bin']
    timesteps_pred = 1

    #training set
    seq_gen = (list(gen_X_sequence(data_train[data_train['ID']==id], sequence_length, seq_cols, timesteps_pred, type_data= 'train')) 
                    for id in data_train['ID'].unique())
    # generate sequences and convert to numpy array
    dbX = np.concatenate(list(seq_gen))

    seq_gen = (list(gen_Y_sequence(data_train[data_train['ID']==id], sequence_length, seq_cols1, timesteps_pred, type_data= 'train')) 
                    for id in data_train['ID'].unique())
    # generate sequences and convert to numpy array
    dbY = np.concatenate(list(seq_gen)).reshape(-1,)

    #val set
    seq_gen = (list(gen_X_sequence(data_val[data_val['ID']==id], sequence_length, seq_cols, timesteps_pred, type_data= 'train')) 
                    for id in data_val['ID'].unique())
    # generate sequences and convert to numpy array
    dbX_val = np.concatenate(list(seq_gen))

    seq_gen = (list(gen_Y_sequence(data_val[data_val['ID']==id], sequence_length, seq_cols1, timesteps_pred, type_data= 'train')) 
                    for id in data_val['ID'].unique())
    # generate sequences and convert to numpy array
    dbY_val = np.concatenate(list(seq_gen)).reshape(-1,)

    #test set
    seq_gen = (list(gen_X_sequence(data_test[data_test['ID']==id], sequence_length, seq_cols, timesteps_pred, type_data= 'train')) 
                    for id in data_test['ID'].unique())
    # generate sequences and convert to numpy array
    dbX_test = np.concatenate(list(seq_gen))

    seq_gen = (list(gen_Y_sequence(data_test[data_test['ID']==id], sequence_length, seq_cols1, timesteps_pred, type_data= 'train')) 
                    for id in data_test['ID'].unique())
    # generate sequences and convert to numpy array
    dbY_test = np.concatenate(list(seq_gen)).reshape(-1,)

    return (
        tf.data.Dataset.from_tensor_slices((dbX, dbY)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((dbX_val, dbY_val)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((dbX_test, dbY_test)).batch(batch_size),
    )


class MyHyperModel(HyperModel):

    def __init__(self, input_shape, output_shape):
        self.input_shape = input_shape
        self.output_shape = output_shape

    def build(self, hp):
        # build model
        input_layer = Input(shape=self.input_shape)

        x = LayerNormalization(axis=1)(input_layer)
        x = SimpleRNN(32, dropout=0, recurrent_dropout=0, return_sequences=True)(x)
        x = SimpleRNN(32, dropout=0, recurrent_dropout=0, return_sequences=True)(x)
        x = SimpleRNN(32, dropout=0, recurrent_dropout=0, return_sequences=False)(x)
        x = Dense(self.output_shape, activation='softmax')(x)
        output_layer = x

        model = Model(input_layer, output_layer)

        # compile model
        model.compile(
            optimizer=Adam(),
            loss='sparse_categorical_crossentropy',
            metrics=['SparseCategoricalAccuracy'])

        return model


# Load training, validation and test data
batch_size = 4096
sequence_length = 30
train_dataset, val_dataset, test_dataset = get_dataset(
    sequence_length=sequence_length, batch_size=batch_size)


# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

# Open a strategy scope.
with strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
    model = build_model(input_shape = (sequence_length, 3), output_shape = 20)

# model_directory = 'CNN_Model3_6' + '/' + '100_structures_adaptiveLR_1'
# model_path = model_directory + '/' + 'CNN_Model3_6_adaptiveLR_1'

# # get model as json string and save to file
# model_as_json = model.to_json()
# with open(model_path + '.json', "w") as json_file:
#     json_file.write(model_as_json)

es = keras.callbacks.EarlyStopping(monitor='sparse_categorical_accuracy', min_delta=0, patience=100, verbose=2, mode='max')
mc = keras.callbacks.ModelCheckpoint('best_model_rnn_prob.h5', monitor='sparse_categorical_accuracy', mode='max', 
                                     save_weights_only=True, save_best_only=True)
# tb = tf.keras.callbacks.TensorBoard(model_directory)

for lr in [0.01]:
    model.compile(
            optimizer=Adam(lr),
            loss='sparse_categorical_crossentropy',
            metrics=['SparseCategoricalAccuracy'])

    # Train the model on all available devices.
    history = model.fit(train_dataset, epochs=500, verbose=2, validation_data=val_dataset, callbacks = [mc])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of devices: 1
Epoch 1/500
14/14 - 8s - loss: 2.8360 - sparse_categorical_accuracy: 0.0785 - val_loss: 2.6725 - val_sparse_categorical_accuracy: 0.1011
Epoch 2/500
14/14 - 2s - loss: 2.6389 - sparse_categorical_accuracy: 0.1082 - val_loss: 2.6338 - val_sparse_categorical_accuracy: 0.1011
Epoch 3/500
14/14 - 2s - loss: 2.6211 - sparse_categorical_accuracy: 0.1084 - val_loss: 2.6247 - val_sparse_categorical_accuracy: 0.1119
Epoch 4/500
14/14 - 2s - loss: 2.6185 - sparse_categorical_accuracy: 0.1109 - val_loss: 2.6238 - val_sparse_categorical_accuracy: 0.1119
Epoch 5/500
14/14 - 2s - loss: 2.6165 - sparse_categorical_accuracy: 0.1106 - val_loss: 2.6224 - val_sparse_categorical_accuracy: 0.1119
Epoch 6/500
14/14 - 2s - loss: 2.6154 - sparse_categorical_accu

KeyboardInterrupt: ignored